In [3]:
%%writefile query1.hql

USE demodb;

DROP TABLE if exists posts_sample_external; 

CREATE EXTERNAL TABLE posts_sample_external 
(year string,
month string)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.RegexSerDe' 
WITH SERDEPROPERTIES (
    "input.regex" = '.*?(?=.*\\bCreationDate=\"(\\d+)-(\\d+)).*'
)
STORED AS TEXTFILE
LOCATION '/data/stackexchange1000/posts/';

set hive.exec.dynamic.partition.mode=nonstrict;
set hive.exec.dynamic.partition=true;
set hive.exec.max.dynamic.partitions=2000;
set hive.exec.max.dynamic.partitions.pernode=1000;


DROP TABLE if exists posts_sample; 

CREATE TABLE posts_sample 
(count int) 
PARTITIONED BY (year string, month string)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t';

FROM posts_sample_external
INSERT OVERWRITE TABLE posts_sample
PARTITION (year, month)
SELECT count(*) as count, year, concat(year,"-",month) as month
GROUP BY year, month;



Overwriting query1.hql


In [4]:
%%writefile query2.hql

USE demodb;

SELECT year, month, count FROM (
    SELECT * FROM (
        SELECT
            count,
            year,
            month,
            ROW_NUMBER() OVER(ORDER BY year, month ASC) as rowid
        FROM posts_sample
    ) AS q1
    ORDER BY rowid ASC
) AS q2
WHERE rowid='3';

Overwriting query2.hql


In [5]:
! hive -S -f query1.hql

In [8]:
! hive -f query2.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 1.155 seconds
Query ID = jovyan_20180716045757_6d137af4-c5d7-446f-a4a9-9a13d708b00c
Total jobs = 2
Launching Job 1 out of 2
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1531693512718_0040, Tracking URL = http://b5a379c75491:8088/proxy/application_1531693512718_0040/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1531693512718_0040
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2018-07-16 04:57:39,761 Stage-1 map = 0%,  reduce = 0%
2018-07-16 04:57:48,599 Stage-1 map = 100%, 